# Exercise 2

## Imports

In [ ]:
import pickle
import os.path
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StringType, ArrayType, FloatType, DoubleType
from itertools import combinations
from typing import Iterable, Any, List, Set

import pandas as pd
import numpy as np
from typing import Union

import matplotlib.pyplot as plt

## Spark initialization

In [ ]:
spark = SparkSession.builder \
    .appName('exercise1') \
    .config('spark.master', 'local[*]') \
    .getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", "16")

## Prepare the data

*Start with the small dataset first.*

In [ ]:
...

## Collaborative Filtering

Calculate baselines.

In [ ]:
...

Calculate similarities.

In [ ]:
...

Get $k$ most similar (clustering perhaps?).

In [ ]:
...

## Testing

Leave out 10% of the ratings for evaluation.

In [ ]:
...

Use evaluation metric on the 10% (precision at 10 / rank correlation?).

In [ ]:
...